In [1]:
# !dir

In [2]:
# !pip install selenium

In [8]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import time

In [51]:
from bs4 import BeautifulSoup as bs
# from hdfs import InsecureClient
import pandas as pd
import folium
from datetime import datetime

In [9]:
# Chrome 옵션
options = Options()
options.binary_location = r'C:/Users/penta/Documents/PythonJupyterlab/chrome/chrome.exe' #크롬 브라우저 경로
# options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
options.add_argument('--disable-gpu')
options.add_argument('--window-size=1920,1080')

#ChromeDriver 설정
service = Service(r'C:/Users/penta/Documents/PythonJupyterlab/chrome/chromedriver.exe')
driver = webdriver.Chrome(service=service, options=options)

In [10]:
driver.get('https://www.starbucks.co.kr/index.do')
print(driver.title)

hover_el = driver.find_element(By.CSS_SELECTOR, '#gnb > div > nav > div > ul > li.gnb_nav03 > h2 > a')
menu_opt = driver.find_element(By.CSS_SELECTOR, '#gnb > div > nav > div > ul > li.gnb_nav03 > div > div > div > ul:nth-child(1) > li.gnb_sub_ttl > a')

# 마우스 호버 액션
time.sleep(3)
actions = ActionChains(driver)
actions.move_to_element(hover_el).perform() # perform == run
print('[[호버 동작]]')
# 대기시간
time.sleep(1.5)
print('[[매장메뉴 선택]]')
menu_opt.click()

print('[[페이지 이동 중]]')
time.sleep(6.5)
driver.find_element(By.CSS_SELECTOR, '.loca_search > h3 > a').click()
time.sleep(0.5)
driver.find_element(By.CSS_SELECTOR, '.sido_arae_box > li:nth-child(6) > a').click()
time.sleep(0.5)
driver.find_element(By.CSS_SELECTOR, '.gugun_arae_box > li:nth-child(1) > a').click()

Starbucks Korea
[[호버 동작]]
[[매장메뉴 선택]]
[[페이지 이동 중]]


In [14]:
# 현재 페이지 정보 가져오기
star_list = []
html = driver.page_source
soup = bs(html, 'html.parser')
contents = soup.select_one('.quickSearchResultBoxSidoGugun')
listnames = contents.select('li')
for listname in listnames:
    star_list.append([listname['data-name'], listname['data-lat'], listname['data-long'], listname.select_one('p').text, listname.select_one('i')['class'][0]])

driver.quit()


In [17]:
columns = ['name', 'lat', 'lon', 'addr', 'stype']
starbucks = pd.DataFrame(star_list, columns=columns)

In [18]:
starbucks.head(10)

,name,lat,lon,addr,stype
0,금정남산동DT,35.2621786292157,129.091325043351,부산광역시 금정구 중앙대로 1989 (남산동)1522-3232,pin_generalDT
1,금정구청DT,35.2388135211636,129.092985849499,부산광역시 금정구 중앙대로 1730 (부곡동)1522-3232,pin_generalDT
2,온천장역DT,35.224006,129.08656,부산광역시 금정구 식물원로 13 (장전동)1522-3232,pin_generalDT
3,부산대정문,35.23212165114605,129.0844959292725,부산광역시 금정구 부산대학로64번길 12 (장전동)1522-3232,pin_general
4,부산대역,35.23052180284867,129.08869433543111,"부산광역시 금정구 장전로12번길 64, 1~4층 (장전동)1522-3232",pin_general
5,금정이마트,35.249819818853915,129.09078227309334,부산광역시 금정구 중앙대로1841번길 24 (구서동)1522-3232,pin_general
6,금정구서,35.246908383445785,129.08873844189125,부산광역시 금정구 금강로 441 (구서동)1522-3232,pin_general
7,부산서동,35.21463924851766,129.10558442984572,부산광역시 금정구 서동로 173 (서동)1522-3232,pin_general
8,용호동W스퀘어,35.1337011,129.113324,"부산광역시 남구 분포로 145 (용호동) 더블유스퀘어동 1019호,1020호,102...",pin_general
9,동명대DT,35.12311959047579,129.09901642703608,부산광역시 남구 신선로 423 (용당동)1522-3232,pin_generalDT


In [26]:
# starbucks.groupby('stype').describe()
starbucks.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 143 entries, 0 to 142
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   name    143 non-null    object
 1   lat     143 non-null    object
 2   lon     143 non-null    object
 3   addr    143 non-null    object
 4   stype   143 non-null    object
dtypes: object(5)
memory usage: 5.7+ KB


In [31]:
timestamp = datetime.now().strftime('%Y%m%d%H%M%S')
print(timestamp)

20241227150749


In [48]:
timestamp = datetime.now().strftime('%Y%m%d%H%M%S')
csv_path = f'starbucks_{timestamp}.csv'
csv_file = starbucks.to_csv(csv_path, index=True)
print(f'변환 완료: {csv_path}')
path = f'./resultData/{csv_path}'

# try:
#     path = f'/user/hadoop/{csv_path}'
#     hdfs = InsecureClient('http://namenode:9870', user='hadoop')
#     hdfs.upload(path, csv_path)
#     print('hdfs 업로드 완료')
# except Exception as e:
#     print(f'hdfs 업로드 실패 - {e}')


변환 완료: starbucks_20241227151939.csv


In [ ]:
try:
    print(120 / 0 * 33)
except Exception as e:
    print(f'오류 발생 - {e}')

In [ ]:
data = starbucks.copy()
coordinates = data[['lat', 'lon']].to_numpy()
coordinates

In [ ]:
place = data[['name']].to_numpy()
place

In [68]:
cn_lat, cn_lon = coordinates[4]
print(cn_lat, cn_lon)
m = folium.Map(location=[cn_lat, cn_lon], zoom_start=11)

for i, (lat, lon) in enumerate(coordinates):
    # print(lat, type(lat))
    folium.CircleMarker(
        location=[lat, lon],
        radius=10,
        color='blue',
        fill=True,
        fill_color='skyblue',
        fill_opacity=0.5,
        tooltip=place[i]
    ).add_to(m)
    # 라벨 추가 (Marker로 위치 조정)
    folium.Marker(
        location=[float(lat) + 0.001, float(lon)],  # 위도 약간 증가
        icon=folium.DivIcon(html=f'<div style="font-size: 9px; color: black; background-color: white; width:50px;white-space: nowrap; overflow: hidden;text-overflow: ellipsis;">{place[i]}</div>')
    ).add_to(m)
# 확인
m

35.23052180284867 129.08869433543111
